Get some transactions
Add plaid merchant, add website

based on https://github.com/meetcleo/data-science/blob/master/transaction-enrichment/notebooks/transactions_categories/T1-24/Data_for_TM_annotations.ipynb



In [17]:
import pandas as pd
import re
import numpy as np
from datetime import datetime
import boto3
from botocore.exceptions import ClientError
from io import StringIO
from cleodata.utils.secrets import get_secret
from cleodata.sources.sync.sync import SyncDataSource
boto3.setup_default_session(profile_name='DataScientist-878877078763')
redshift_source = SyncDataSource("data_exploration", use_redshift=True, redshift_cluster="cleo-production-redshift", redshift_db="cleo")

2024-05-31 14:02:07 [debug    ] fetching credentials          
2024-05-31 14:02:09 [info     ] Credentials acquired          
2024-05-31 14:02:09 [info     ] Built connection pool         


In [2]:
%pip install s3fs

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install fastparquet

Note: you may need to restart the kernel to use updated packages.


In [4]:
import s3fs
from fastparquet import ParquetFile

In [5]:
def read_from_s3(path):
    """Read parquet files and combine them into a single dataframe"""
    fs = s3fs.core.S3FileSystem()
    all_paths_from_s3 = fs.glob(path=f"{path}*.parquet")

    if len(all_paths_from_s3) > 0:
        s3 = s3fs.S3FileSystem()
        fp_obj = ParquetFile(
            all_paths_from_s3, open_with=s3.open
        )  # use s3fs as the filesystem
        data = fp_obj.to_pandas()
        return data
    elif len(all_paths_from_s3)==1:
        return pd.read_parquet(all_paths_from_s3[0])
    else:
        print(f"Nothing found")
        print(f"paths from a{all_paths_from_s3}")
    
def read_csv_s3(bucket, key):
    try:
        s3 = boto3.client('s3')
        obj = s3.get_object(Bucket=bucket, Key=key)
        df = pd.read_csv(obj['Body'])
        return df
    except ClientError as ex:
        if ex.response['Error']['Code'] == 'NoSuchKey':
            print("Key doesn't match. Please check the key value entered.")


def list_s3_flies(base_path):
    fs = s3fs.core.S3FileSystem()
    all_paths_from_s3 = fs.glob(path=f"{base_path}*.parquet")
    return all_paths_from_s3

 - **Description 1**: Text description of transactions: Internal description or processed and cleared description from external provider 

 - **Description 2**: Text description of transactions, original from external provider. The text returned by the financial institution to describe the transaction. 

 - **Additional Merchant name (original):** Merchant name of transaction from external provider 

 - **Original Transfer counterparty name:** Extracted from some transaction information from external provider 

 - **Location:** If not empty, will include information as in following example:

{'lat': 28.607128, 'lon': -81.387253, 'city': 'Orlando', 'region': 'FL', 'address': '609 Lee Rd', 'country': None, 'postal_code': '32810', 'store_number': None}

- **Payment channel:** The channel used to make a payment. Possible values: online, in store, other: - online: transactions that took place online. - in store: transactions that were made at a physical location. - other: transactions that relate to banks, e.g. fees or deposits. 

- **Payee:** name of transfer receiver -+ Payer: name of transfer sender

- Google search link for “Additional Merchant name (original)”

- Google search link for “Description 1”

- Google search link for “Description 2”

- Google search link for “Original Transfer counterparty name”

- Google search link for “Description 2 and Original Transfer counterparty name”


In [6]:
#original query
# df = redshift_source.fetch_data( """
# with 
# presect_transactions as (
#     select 
#             tt.id,
#             tt.transaction_category_id,
#             tt.currency_code,
#             tt.amount,
#             tt.corrected_made_on, 
#             tt.made_on, 
#             tt.login_provider_additional_attributes,
#             tt.description,
#             tt.status
#     from transactions tt
#     where tt.made_on between '2024-01-20' and '2024-01-31'
#     limit 10000
# )

# select
#     tt.id,
#     tc.name as transaction_category_v1,
#     json_extract_path_text(tt.login_provider_additional_attributes, 'personal_finance_category', 'detailed') 
#         as plaid_category_detailed,
#     m.name as merchant_name,
#     case when td.payee!='None' and td.payee is not null then td.payee
#         when td.payer!='None' and td.payer is not null then td.payer
#         else Null end as transfer_counterparty_name,

#     coalesce(tt.corrected_made_on, tt.made_on) as transaction_date,

#     case when tt.amount < 0 then 'Out'
#          when tt.amount > 0 then 'In'
#          else 'Zero'
#         end as direction, 
#     abs(tt.amount) as amount, 
#     tt.currency_code,
    
        
#     tt.description as description_1,
#     json_extract_path_text(tt.login_provider_additional_attributes, 'original_description') 
#         as description_2,
#     json_extract_path_text(tt.login_provider_additional_attributes, 'merchant_name') 
#         as additional_merchant_name,
#     td.company as original_transfer_counterparty_name,
    
#     td.payee,
#     td.payer,
    
#     json_extract_path_text(tt.login_provider_additional_attributes, 'payment_channel') 
#         as payment_channel,
#     json_extract_path_text(tt.login_provider_additional_attributes, 'location') 
#         as location,
#     tt.status

# from  presect_transactions tt
# left join transaction_categories tc on tc.id = tt.transaction_category_id
# left join transaction_descriptions td on td.transaction_id = tt.id
# left join merchants m on m.id = td.merchant_id and td.merchant_id is not null
# """)

In [7]:
sql_trans_with_merchant = """ 
with tt as (select
        tt.id,
        tt.amount,
        tt.corrected_made_on,
        tt.description,
        json_extract_path_text(tt.login_provider_additional_attributes, 'original_description')
        as original_description_plaid,
        json_extract_path_text(tt.login_provider_additional_attributes, 'merchant_name')
        as merchant_name_plaid,
        json_extract_path_text(tt.login_provider_additional_attributes, 'personal_finance_category', 'detailed')
        as category_detailed_plaid,
        json_extract_path_text(tt.login_provider_additional_attributes, 'personal_finance_category', 'primary')
        as category_primary_plaid,
        tt.made_on,
        tt.currency_code,

        json_extract_path_text(tt.login_provider_additional_attributes, 'payment_channel')
        as payment_channel,
        json_extract_path_text(tt.login_provider_additional_attributes, 'payment_method')
        as payment_method,
        json_extract_path_text(tt.login_provider_additional_attributes, 'payment_processor_method')
        as payment_processor_method,
        json_extract_path_text(tt.login_provider_additional_attributes, 'location','city')
        as city,
        json_extract_path_text(tt.login_provider_additional_attributes, 'location','region')
        as region,
        json_extract_path_text(tt.login_provider_additional_attributes, 'location','country')
        as country,
        tt.transaction_category_id,
        tt.login_provider_additional_attributes,
        tt.status
    from transactions tt
    where tt.made_on between '2024-03-02' and '2024-03-03'
    limit 100 )

select  tt.id,
        tt.amount,
        coalesce(tt.corrected_made_on, tt.made_on) as transaction_date,
        tt.corrected_made_on,
        tt.description,
        json_extract_path_text(tt.login_provider_additional_attributes, 'original_description')
        as original_description_plaid,
        json_extract_path_text(tt.login_provider_additional_attributes, 'merchant_name')
        as merchant_name_plaid,
        mm.name as merchant_name,
        mm.website as merchant_website,
        mm.twitter_name,
        mm.latitude as merchant_latitude,
        mm.longitude as merchant_longitude,

        json_extract_path_text(tt.login_provider_additional_attributes, 'personal_finance_category', 'detailed')
        as category_detailed_plaid,
        json_extract_path_text(tt.login_provider_additional_attributes, 'personal_finance_category', 'primary')
        as category_primary_plaid,
        tt.made_on,
        tt.currency_code,

        json_extract_path_text(tt.login_provider_additional_attributes, 'payment_channel')
        as payment_channel,
        json_extract_path_text(tt.login_provider_additional_attributes, 'payment_method')
        as payment_method,
        json_extract_path_text(tt.login_provider_additional_attributes, 'payment_processor_method')
        as payment_processor_method,
        json_extract_path_text(tt.login_provider_additional_attributes, 'location','city')
        as city,
        json_extract_path_text(tt.login_provider_additional_attributes, 'location','region')
        as region,
        json_extract_path_text(tt.login_provider_additional_attributes, 'location','country')
        as country,
        tt.transaction_category_id,
        tt.login_provider_additional_attributes,
        tt.status,
        transaction_descriptions.merchant_id,
        case when transaction_descriptions.payee!='None' and transaction_descriptions.payee is not null then transaction_descriptions.payee
            when transaction_descriptions.payer!='None' and transaction_descriptions.payer is not null then transaction_descriptions.payer
            else Null end as transfer_counterparty_name
from tt
left join  transaction_descriptions on tt.id = transaction_descriptions.transaction_id
left join merchants mm on mm.id = transaction_descriptions.merchant_id

"""

In [8]:
start_date_s = '2024-05-13'
end_date_s = '2024-05-13'
date_range = pd.date_range(start=start_date_s, end=end_date_s)
# Convert the date range to a list of strings
date_list = date_range.strftime('%Y-%m-%d').tolist()
date_list

['2024-05-13']

In [9]:
def get_query(start_date_s,end_date_s):

  sql_fct_trans_with_merchant = f"""  
  select ftt.transaction_id,
        ftt.corrected_made_on,
        ftt.amount,
        ftt.currency_code,
        ftt.description,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'original_description')
          as original_description_plaid,
        ftt.merchant_name,
        json_extract_path_text(ftt.login_provider_additional_attributes, 'merchant_name') as merchant_name_plaid,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'transaction_type')
          as transaction_type_plaid,
        json_extract_path_text(ftt.login_provider_additional_attributes, 'counterparties', '0', 'name') AS counterparty_name,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'counterparties', '0', 'type') AS counterparty_type,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'counterparties', '0', 'website') AS counterparty_website,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'counterparties', '0', 'logo_url') AS counterparty_logo_url,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'counterparties', '0', 'entity_id') AS counterparty_entity_id,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'counterparties', '0', 'confidence_level') AS counterparty_confidence_level,
        ftt.merchant_id,
        ftt.company,
          json_extract_path_text(ftt.login_provider_additional_attributes, 'personal_finance_category', 'detailed')
          as category_detailed_plaid,
          json_extract_path_text(ftt.login_provider_additional_attributes, 'personal_finance_category', 'primary')
          as category_primary_plaid,
          json_extract_path_text(ftt.login_provider_additional_attributes, 'payment_channel')
          as payment_channel,
          json_extract_path_text(ftt.login_provider_additional_attributes, 'payment_method')
          as payment_method,
          json_extract_path_text(ftt.login_provider_additional_attributes, 'payment_processor_method')
          as payment_processor_method,
          json_extract_path_text(ftt.login_provider_additional_attributes, 'location','city')
          as city,
          json_extract_path_text(ftt.login_provider_additional_attributes, 'location','region')
          as region,
          json_extract_path_text(ftt.login_provider_additional_attributes, 'location','country')
          as country,
          ftt.status,
        ftt.decline_code,
        ftt.login_provider_additional_attributes
  from analytics.fct_transactions ftt
  where corrected_made_on >= '{start_date_s}' and corrected_made_on <= '{end_date_s}'
  and original_description_plaid is not null
  and original_description_plaid != ''
  and original_description_plaid != ' '
  and ABS(amount) > 0.1
  --limit 5000000
  """
  return sql_fct_trans_with_merchant

In [10]:
print(get_query(start_date_s,end_date_s))

  
  select ftt.transaction_id,
        ftt.corrected_made_on,
        ftt.amount,
        ftt.currency_code,
        ftt.description,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'original_description')
          as original_description_plaid,
        ftt.merchant_name,
        json_extract_path_text(ftt.login_provider_additional_attributes, 'merchant_name') as merchant_name_plaid,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'transaction_type')
          as transaction_type_plaid,
        json_extract_path_text(ftt.login_provider_additional_attributes, 'counterparties', '0', 'name') AS counterparty_name,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'counterparties', '0', 'type') AS counterparty_type,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'counterparties', '0', 'website') AS counterparty_website,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'counterp

In [11]:
run_one_off = 1

for one_date in date_list:
    start_date = one_date
    end_date = one_date
    if run_one_off==0 :
        df_trans_orig = redshift_source.fetch_data(sql_fct_trans_with_merchant)
    elif run_one_off == 1:
        dataset_id = datetime.now().strftime("%y%m%d_%H%M")
        sql_fct_trans_with_merchant = get_query(start_date, end_date)
        sql = f"""
        unload ('select * from ({sql_fct_trans_with_merchant.replace("'", "''")}
        )')
            to 's3://cleo-data-science/transaction_enrichment/experimental_data/caste/raw/trans_{start_date}_{end_date}'
            iam_role 'arn:aws:iam::878877078763:role/CleoRedshiftToS3'
            format as parquet
            cleanpath;"""
        print(sql)
        redshift_source.execute_query(sql)
        
        path_file = f"s3://cleo-data-science/transaction_enrichment/experimental_data/caste/raw/trans_{start_date}_{end_date}"
        print(path_file)
        #df_trans = read_from_s3(path_file)

        # bucket_name = path_file.split('://')[1].split('/')[0]
        # key = path_file.split('://')[1].split(bucket_name)[1][1:]
        # print(f"bucket {bucket_name}")
        # print(f"key {key}")
        # df_trans  = read_csv_s3(bucket_name, key )
    elif run_one_off ==2:
        path_file = "s3://cleo-data-science/transaction_enrichment/experimental_data/caste/raw/trans_2024-05-20_2024-05-20"
        df_trans = read_from_s3(path_file)


        unload ('select * from (  
  select ftt.transaction_id,
        ftt.corrected_made_on,
        ftt.amount,
        ftt.currency_code,
        ftt.description,
      json_extract_path_text(ftt.login_provider_additional_attributes, ''original_description'')
          as original_description_plaid,
        ftt.merchant_name,
        json_extract_path_text(ftt.login_provider_additional_attributes, ''merchant_name'') as merchant_name_plaid,
      json_extract_path_text(ftt.login_provider_additional_attributes, ''transaction_type'')
          as transaction_type_plaid,
        json_extract_path_text(ftt.login_provider_additional_attributes, ''counterparties'', ''0'', ''name'') AS counterparty_name,
      json_extract_path_text(ftt.login_provider_additional_attributes, ''counterparties'', ''0'', ''type'') AS counterparty_type,
      json_extract_path_text(ftt.login_provider_additional_attributes, ''counterparties'', ''0'', ''website'') AS counterparty_website,
      json_extract_path

In [12]:
path_file = "s3://cleo-data-science/transaction_enrichment/experimental_data/caste/raw/trans_2024-05-13_2024-05-13"

In [13]:
df_trans = read_from_s3(path_file)
df_trans.shape

(8681203, 28)

In [14]:
df_trans.head(5)

,transaction_id,corrected_made_on,amount,currency_code,description,original_description_plaid,merchant_name,merchant_name_plaid,transaction_type_plaid,counterparty_name,...,category_primary_plaid,payment_channel,payment_method,payment_processor_method,city,region,country,status,decline_code,login_provider_additional_attributes
0,9815920130,2024-05-13,300.00,USD,Atm Deposit Us Bank Hermitaghermitage Tnus1 05...,Atm Deposit Us Bank Hermitaghermitage Tnus1 05...,None,,special,U.S. Bank,...,TRANSFER_IN,other,,,,,,None,,"{""datetime"": ""2024-05-13T00:00:00.000Z"", ""loca..."
1,9815920158,2024-05-13,-18.60,USD,Recurring Debit Purchase Apple.com/bill 05/12 ...,Recurring Debit Purchase Apple.com/bill 866-71...,Apple,Apple,digital,Apple,...,GENERAL_MERCHANDISE,online,,,,,,None,,"{""datetime"": ""2024-05-13T00:00:00.000Z"", ""loca..."
2,9815920131,2024-05-13,-100.00,USD,Atm Withdrawal Us Bank Hermitaghermitage Tnus1...,Atm Withdrawal Us Bank Hermitaghermitage Tnus1...,Cash Withdrawal,,special,U.S. Bank,...,TRANSFER_OUT,other,,,,,,None,,"{""datetime"": ""2024-05-13T00:00:00.000Z"", ""loca..."
3,9815920132,2024-05-13,-20.00,USD,Atm Withdrawal Us Bank Hermitaghermitage Tnus1...,Atm Withdrawal Us Bank Hermitaghermitage Tnus1...,Cash Withdrawal,,special,U.S. Bank,...,TRANSFER_OUT,other,,,,,,None,,"{""datetime"": ""2024-05-13T00:00:00.000Z"", ""loca..."
4,9815920133,2024-05-13,-20.00,USD,Atm Withdrawal Us Bank Hermitaghermitage Tnus1...,Atm Withdrawal Us Bank Hermitaghermitage Tnus1...,Cash Withdrawal,,special,U.S. Bank,...,TRANSFER_OUT,other,,,,,,None,,"{""datetime"": ""2024-05-13T00:00:00.000Z"", ""loca..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8681198,9970242706,2024-05-13,-5.38,USD,Debit PIN Purchase FOOD EXPRESS MO TOBYHANNA 3...,Debit PIN Purchase FOOD EXPRESS MO TOBYHANNA 3...,Food Express,Food Express,place,Food Express,...,FOOD_AND_DRINK,in store,,,Hanna,MO,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."
8681199,9970242708,2024-05-13,-3.75,USD,Mobile Purchase Sign Based 05/09 09:10a #8315 ...,Mobile Purchase Sign Based 05/09 09:10a #8315 ...,Secaucus Train Statqps,Secaucus Train Stat,place,Secaucus Train Stat,...,TRANSPORTATION,online,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."
8681200,9970242707,2024-05-13,-8.83,USD,Lyft,Mobile Purchase Sign Based 05/08 08:59a #8315 ...,Lyft,Lyft,special,Lyft,...,TRANSPORTATION,online,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."
8681201,9970242700,2024-05-13,-50.00,USD,ACH Electronic Debit - GREENLIGHT APP GREENLIGHT,ACH Electronic Debit - GREENLIGHT APP GREENLIGHT,Greenlight,,place,Greenlight,...,TRANSFER_OUT,other,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."


In [15]:
path_file_processed = path_file.split('raw/')[0]+'processed/'+path_file.split('raw/')[1]
path_file_processed


's3://cleo-data-science/transaction_enrichment/experimental_data/caste/processed/trans_2024-05-13_2024-05-13'

If merchant name is not present, user merchant_name_plaid. 
If merchant_name is not present, use counterparty_name



In [18]:
# Coalescing merchant names

#replace None with null
#replace Nan
# Replace empty spaces, None, and strings with only spaces with NaN
df_trans['merchant_name'] = df_trans['merchant_name'].replace(r'^\s*$', np.nan, regex=True)
df_trans['merchant_name_plaid'] = df_trans['merchant_name_plaid'].replace(r'^\s*$', np.nan, regex=True)


df_trans['merchant_name'] = df_trans['merchant_name'].map({'None':None})
df_trans['merchant_name_plaid'] = df_trans['merchant_name_plaid'].map({'':None,' ':None})
# create a column combined_merchant where we take the any merchant name : Cleo, or plaid, or counterparty 
df_trans['merchant_name_combined'] = df_trans['merchant_name'].combine_first(df_trans['merchant_name_plaid'])
df_trans['merchant_name_combined'] = df_trans['merchant_name_combined'].combine_first(df_trans['counterparty_name'])
# Remove data without merchant name for training data
df_trans = df_trans[(~df_trans['merchant_name_combined'].isnull()) & ~df_trans['merchant_name_combined'].isin(['',' '])][:]
df_trans['merchant_name_combined_len'] = df_trans['merchant_name_combined'].apply(lambda x: len(x))
df_trans = df_trans[df_trans['merchant_name_combined_len']>=1]
df_trans.drop('merchant_name_combined_len', axis=1, inplace=True)

# Coalescing descriptions 
# if original_description_plaid is empty use description
df_trans['description_combined'] = df_trans['original_description_plaid'].combine_first(df_trans['description'])
df_trans['len_description'] = df_trans['description_combined'].apply(lambda x: len(x))
df_trans = df_trans[df_trans['len_description'] >=2]
df_trans.drop('len_description', axis=1, inplace=True)
##
#replace 'other' with ''
df_trans['payment_channel_processed'] = df_trans['payment_channel'].apply(lambda x: 'None' if x == 'other' else x)
# do some light processing to make strings shorter
df_trans['original_description_plaid_processed'] =  df_trans['original_description_plaid'].apply(lambda x: re.sub('\\\\+','\\\\',x))
df_trans['original_description_plaid_processed'] =  df_trans['original_description_plaid_processed'].apply(lambda x: re.sub(r'\d{4,}', ' ', x))

# #if there are 4 or more integers replace with a space
# output_string = re.sub(r'\d{4,}', ' ', df_data_raw['sentence'][1000050])
df_trans['original_description_plaid_processed'] =  df_trans['original_description_plaid_processed'].apply(lambda x: re.sub(' +',' ',x))
df_trans.reset_index(drop=True, inplace=True)

# examples where the description and the merchant name are the same are probably not too informative
df_trans = df_trans[df_trans['merchant_name_combined']!=df_trans['original_description_plaid']]


In [19]:
# There are some cases where Chime is the merchant but it isn't mentioned in the description, so remove these. I am sure there are more like these, we would need to see
df_trans['Chime in descr'] = df_trans['original_description_plaid'].apply(lambda x: 'Chime' in x)

df_trans = df_trans[(df_trans['Chime in descr'] & (df_trans['merchant_name_combined']=='Chime')) | (df_trans['merchant_name_combined']!='Chime')]
df_trans.reset_index(drop=True, inplace=True)

In [20]:
df_trans_no_merchant = df_trans[(df_trans['merchant_name_combined'].isnull()) | df_trans['merchant_name_combined'].isin(['',' '])]
df_trans_no_merchant

,transaction_id,corrected_made_on,amount,currency_code,description,original_description_plaid,merchant_name,merchant_name_plaid,transaction_type_plaid,counterparty_name,...,country,status,decline_code,login_provider_additional_attributes,merchant_name_combined,description_combined,len_description,payment_channel_processed,original_description_plaid_processed,Chime in descr


In [21]:
print(df_trans.shape)

(7163494, 34)


In [22]:
# examples where the description and the merchant name are the same are probably not too informative
df_trans_cln3 = df_trans[df_trans['merchant_name_combined']!=df_trans['original_description_plaid']]
print(df_trans_cln3.shape)
#df_trans_cln3['merchant_name_combined'].value_counts()[:-40]

(6709646, 34)


In [23]:
def look_up_for_one_merchant(merchant_name, df):
    return df[df['merchant_name_combined']==merchant_name]

In [24]:
look_up_for_one_merchant('Affirm', df_trans_cln3)

,transaction_id,corrected_made_on,amount,currency_code,description,original_description_plaid,merchant_name,merchant_name_plaid,transaction_type_plaid,counterparty_name,...,country,status,decline_code,login_provider_additional_attributes,merchant_name_combined,description_combined,len_description,payment_channel_processed,original_description_plaid_processed,Chime in descr
417,9817460216,2024-05-13,-190.00,USD,PURCHASE 0510 AFFIRM.COM PAYMEN SAN FRANCISCOC...,PURCHASE 0510 AFFIRM.COM PAYMEN SAN FRANCISCOC...,NaN,NaN,place,Affirm,...,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm,PURCHASE 0510 AFFIRM.COM PAYMEN SAN FRANCISCOC...,71,None,PURCHASE AFFIRM.COM PAYMEN SAN FRANCISCOCA XXX...,False
1580,9821978880,2024-05-13,-17.90,USD,AFFIRM INC,AFFIRM INC,NaN,NaN,place,Affirm,...,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm,AFFIRM INC,10,None,AFFIRM INC,False
2214,9824173149,2024-05-13,-38.02,USD,VISA DDA PUR AP 469216 AFFIRM PAY VA1DFMNZ *,VISA DDA PUR AP 469216 AFFIRM PAY VA1DFM...,NaN,NaN,place,Affirm,...,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm,VISA DDA PUR AP 469216 AFFIRM PAY VA1DFM...,73,None,VISA DDA PUR AP AFFIRM PAY VA1DFMNZ 855 423 * CA,False
2217,9824173131,2024-05-13,-42.02,USD,AFFIRM INC AFFIRM PAY,AFFIRM INC AFFIRM PAY,NaN,NaN,place,Affirm,...,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm,AFFIRM INC AFFIRM PAY,27,None,AFFIRM INC AFFIRM PAY,False
2331,9824696260,2024-05-13,-10.26,USD,Affirm P,Affirm P,NaN,NaN,place,Affirm,...,,None,,"{""datetime"": ""2024-05-13T16:26:00.000Z"", ""loca...",Affirm,Affirm P,8,None,Affirm P,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7161571,9836017420,2024-05-13,-15.18,USD,AFFIRM PAY AFFIRM INC,AFFIRM PAY AFFIRM INC,NaN,NaN,place,Affirm,...,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm,AFFIRM PAY AFFIRM INC,21,None,AFFIRM PAY AFFIRM INC,False
7161572,9836017421,2024-05-13,-15.91,USD,AFFIRM PAY AFFIRM INC,AFFIRM PAY AFFIRM INC,NaN,NaN,place,Affirm,...,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm,AFFIRM PAY AFFIRM INC,21,None,AFFIRM PAY AFFIRM INC,False
7161784,9956092328,2024-05-13,-32.82,USD,VISA DDA PUR AP 407283 TRUEACCORD AFFIRM *,VISA DDA PUR AP 407283 TRUEACCORD AFFIRM ...,NaN,NaN,place,Affirm,...,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm,VISA DDA PUR AP 407283 TRUEACCORD AFFIRM ...,73,None,VISA DDA PUR AP TRUEACCORD AFFIRM 866 611 * KS,False
7161794,9956092318,2024-05-13,-10.83,USD,AFFIRM INC AFFIRM PAY,AFFIRM INC AFFIRM PAY,NaN,NaN,place,Affirm,...,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm,AFFIRM INC AFFIRM PAY,27,None,AFFIRM INC AFFIRM PAY,False


In [25]:
df_trans_cln3.columns

Index(['transaction_id', 'corrected_made_on', 'amount', 'currency_code',
       'description', 'original_description_plaid', 'merchant_name',
       'merchant_name_plaid', 'transaction_type_plaid', 'counterparty_name',
       'counterparty_type', 'counterparty_website', 'counterparty_logo_url',
       'counterparty_entity_id', 'counterparty_confidence_level',
       'merchant_id', 'company', 'category_detailed_plaid',
       'category_primary_plaid', 'payment_channel', 'payment_method',
       'payment_processor_method', 'city', 'region', 'country', 'status',
       'decline_code', 'login_provider_additional_attributes',
       'merchant_name_combined', 'description_combined', 'len_description',
       'payment_channel_processed', 'original_description_plaid_processed',
       'Chime in descr'],
      dtype='object')

In [26]:
columns_to_keep = ['transaction_id','corrected_made_on','amount','original_description_plaid','merchant_name_combined','payment_channel']

In [27]:
df_trans_cln4 = df_trans_cln3.drop_duplicates(subset=['original_description_plaid','merchant_name_combined'])

In [28]:
#pre-processing about halves the volume of data
df_trans_cln4.shape

(4616228, 34)

In [29]:
nunique_merchants = df_trans_cln4['merchant_name_combined'].nunique()
print(f"There are {nunique_merchants}  in the dataset")

There are 492815  in the dataset


In [30]:
df_trans_cln4.value_counts(dropna=False)

transaction_id  corrected_made_on  amount  currency_code  description                       original_description_plaid                                                                                                   merchant_name  merchant_name_plaid  transaction_type_plaid  counterparty_name  counterparty_type  counterparty_website  counterparty_logo_url                                     counterparty_entity_id                 counterparty_confidence_level  merchant_id  company  category_detailed_plaid                 category_primary_plaid  payment_channel  payment_method  payment_processor_method  city         region  country  status  decline_code  login_provider_additional_attributes                                                                                                                                                                                                                                                                                                              

In [31]:
df_trans_cln4[columns_to_keep]

,transaction_id,corrected_made_on,amount,original_description_plaid,merchant_name_combined,payment_channel
0,9815920130,2024-05-13,300.00,Atm Deposit Us Bank Hermitaghermitage Tnus1 05...,U.S. Bank,other
1,9815920158,2024-05-13,-18.60,Recurring Debit Purchase Apple.com/bill 866-71...,Apple,online
2,9815920131,2024-05-13,-100.00,Atm Withdrawal Us Bank Hermitaghermitage Tnus1...,U.S. Bank,other
3,9815920132,2024-05-13,-20.00,Atm Withdrawal Us Bank Hermitaghermitage Tnus1...,U.S. Bank,other
4,9815920133,2024-05-13,-20.00,Atm Withdrawal Us Bank Hermitaghermitage Tnus1...,U.S. Bank,other
...,...,...,...,...,...,...
7163488,9970242705,2024-05-13,-15.98,Debit Card Purchase 05/09 07:47p #8315 BUTLER ...,Butler''s Liquor Store,in store
7163489,9970242706,2024-05-13,-5.38,Debit PIN Purchase FOOD EXPRESS MO TOBYHANNA 3...,Food Express,in store
7163490,9970242708,2024-05-13,-3.75,Mobile Purchase Sign Based 05/09 09:10a #8315 ...,Secaucus Train Stat,online
7163491,9970242707,2024-05-13,-8.83,Mobile Purchase Sign Based 05/08 08:59a #8315 ...,Lyft,online


In [32]:
# save this dataset and depending on the type of algorithm you can choose how to reorder the data

df_trans_cln4.to_parquet(path_file_processed)

In [ ]:
print(f"Finsihed writing file {path_file_processed}")

In [ ]:
#df_trans_cln3.to_csv('/Users/claracastellanos/Documents/DATA/MERCHANTS/2024_05_20.csv')

In [ ]:
#load data back in to make sure everythin is ok
df_tmp = read_from_s3("s3://cleo-data-science/transaction_enrichment/experimental_data/caste/biencoder_test1/data1")

In [ ]:
df_tmp.shape

(956812, 30)

In [ ]:
df_tmp

,transaction_id,corrected_made_on,amount,currency_code,description,original_description_plaid,merchant_name,merchant_name_plaid,transaction_type_plaid,counterparty_name,...,payment_method,payment_processor_method,city,region,country,status,decline_code,login_provider_additional_attributes,merchant_name_combined,Chime in descr
index,,,,,,,,,,,,,,,,,,,,,
0,9848923081,2024-05-20,25.00,USD,ZELLE P2P PAYMENT RECEIVED ON 05/19 \,ZELLE P2P PAYMENT RECEIVED ON 05/19 \\,None,None,special,Zelle,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Zelle,False
1,9848926035,2024-05-20,-7.86,USD,McDonald''s,"Purchase / McDonalds 35052 142-3328095, TN (4633)",None,None,place,McDonald''s,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",McDonald''s,False
2,9848926034,2024-05-20,-7.86,USD,McDonald''s,"Purchase / McDonalds 35052 142-3328095, TN (4633)",None,None,place,McDonald''s,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",McDonald''s,False
3,9848945525,2024-05-20,19.65,USD,Transfer from Cash App,Transfer from Cash App,None,None,special,Cash App,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Cash App,False
4,9848948208,2024-05-20,-100.00,USD,POS Withdrawal (FIS) Q NAILS Q NAILS NEWPORT T...,POS Withdrawal (FIS) Q NAILS Q NAILS NEWPORT T...,None,None,place,Q Nails,...,,,Newport,TN,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Q Nails,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170552,9857528533,2024-05-20,-63.27,USD,Cash App*Anthony Cenic,Cash App*Anthony Cenic,None,None,special,Cash App,...,,,,,,None,,"{""datetime"": ""2024-05-20T19:38:03.000Z"", ""loca...",Cash App,False
1170554,9857528559,2024-05-20,-34.70,USD,Point Of Sale Withdrawal 445488848992 DD DOORD...,Point Of Sale Withdrawal 445488848992 DD DOORD...,None,None,place,Mamichuyt,...,,,,CA,US,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Mamichuyt,False
1170555,9857528629,2024-05-20,-10.28,USD,Transfer to Chime Savings Account,Transfer to Chime Savings Account,None,None,special,Chime,...,,,,,,None,,"{""datetime"": ""2024-05-20T19:11:52.000Z"", ""loca...",Chime,True


In [ ]:
df_trans_cln3.to_parquet()